In [0]:
"""
Determine the user_ids and corresponding page_ids of the pages liked by their friend but not by user itself yet
"""

from pyspark.sql.functions import *
from pyspark.sql.window import Window

friends = spark.createDataFrame([
    (1, 2),
    (1, 3),
    (1, 4),
    (2, 1),
    (3, 1),
    (3, 4),
    (4, 1),
    (4, 3)
], ["user_id", "friend_id"])

likes = spark.createDataFrame([
    (1, 'A'),
    (1, 'B'),
    (1, 'C'),
    (2, 'A'),
    (3, 'B'),
    (3, 'C'),
    (4, 'B')
], ["user_id", "page_id"])

friends.show()
likes.show()

+-------+---------+
|user_id|friend_id|
+-------+---------+
|      1|        2|
|      1|        3|
|      1|        4|
|      2|        1|
|      3|        1|
|      3|        4|
|      4|        1|
|      4|        3|
+-------+---------+

+-------+-------+
|user_id|page_id|
+-------+-------+
|      1|      A|
|      1|      B|
|      1|      C|
|      2|      A|
|      3|      B|
|      3|      C|
|      4|      B|
+-------+-------+



In [0]:
friends.createOrReplaceTempView("friends_tbl")
likes.createOrReplaceTempView("likes_tbl")

spark.sql("""
          SELECT f.*, l.page_id FROM friends_tbl f
          INNER JOIN likes_tbl l ON f.friend_id = l.user_id
          LEFT JOIN likes_tbl l1 ON f.user_id = l1.user_id AND l.page_id = l1.page_id 
          WHERE l1.page_id IS NULL
          """).show()


+-------+---------+-------+
|user_id|friend_id|page_id|
+-------+---------+-------+
|      2|        1|      B|
|      2|        1|      C|
|      3|        1|      A|
|      4|        1|      A|
|      4|        1|      C|
|      4|        3|      C|
+-------+---------+-------+

